# Sprint 4 — Webinar 14 (Práctica)

**Modalidad:** Coding Together + Breakrooms  
**Tema:** CTEs de embudo y métricas de retención (SQLite)  
**Versión:** 2025-11-09T10:00:00

> En este webinar continuamos el trabajo de funnels y simulaciones, pero ahora
> nos enfocamos en **construir embudos con CTEs** y **calcular retención por país**
> usando SQLiteOnline.


### Agenda – Webinar 14 (Práctico)

| Hora | Actividad | Descripción |
|------|------------|-------------|
| 00:00 – 00:10 | **Introducción y repaso del Webinar 13** | Revisión de los conceptos previos: embudo de conversión, CTEs y unión de etapas. |
| 00:10 – 00:25 | **Ejercicio 0 (Breakroom colaborativo)** | En grupos pequeños, diseñar una consulta base para identificar usuarios activos por etapa y mes. |
| 00:25 – 00:50 | **Consultas avanzadas con CTEs** | Crear CTEs anidadas para calcular usuarios recurrentes, tasas de retención y abandono. |
| 00:50 – 01:10 | **Análisis comparativo de cohortes** | Medir desempeño mensual de usuarios por cohorte usando funciones de ventana y filtros temporales. |
| 01:10 – 01:20 | **Visualización de resultados** | Interpretar los resultados y validar consistencia del embudo y tasas de conversión. |
| 01:20 – 01:25 | **Mini–Kahoot o quiz SQL** | Preguntas rápidas sobre CTEs, cohortes y funciones analíticas. |
| 01:25 – 01:30 | **Cierre y conclusiones** | Revisión de aprendizajes clave y preparación del próximo módulo teórico. |


## 0. Objetivos de aprendizaje

Al final de este webinar deberías ser capaz de:

1. Construir **CTEs por etapa** de un funnel de eventos y encadenarlas con `LEFT JOIN`.  
2. Calcular **conteos de usuarios por etapa** y **conversiones %** desde la primera visita.  
3. **Segmentar embudos por país** para detectar dónde se pierden más usuarios.  
4. Calcular **usuarios activos acumulados** a D7, D14, D21 y D28 por país.  
5. Convertir esos conteos en **porcentajes de retención** por país.


## 0.1. Cómo usar este notebook con SQLite Online

Trabajaremos con **SQLiteOnline** (o herramienta similar) en lugar de conectar Python a una BD.

Pasos recomendados:

1. Abre tu herramienta online de SQLite.  
2. Selecciona el motor **SQLite**.  
3. Copia y pega los bloques de código SQL que verás en las celdas de tipo `code`.
4. Ejecuta en este orden:
   - Bloque de **DDL** (creación de tablas).  
   - Bloque de **INSERTs** (carga de datos de prueba).  
   - Los bloques de **ejercicios**, uno por uno.

Puedes ir marcando en el notebook qué ejercicios completaste y qué dudas te quedan.


## 1. Recordatorio teórico — Funnels por eventos y retención

### 1.1 Funnels basados en eventos

En un funnel típico de e-commerce cada usuario genera **eventos** como:

- `first_visit` — primera visita al sitio.  
- `select_item` / `select_promotion` — elige un producto u oferta.  
- `add_to_cart` — agrega un ítem al carrito.  
- `begin_checkout` — inicia el proceso de checkout.  
- `add_shipping_info` — completa datos de envío.  
- `add_payment_info` — completa datos de pago.  
- `purchase` — compra completada.

Cada fila de la tabla de eventos suele ser:

- un `user_id`,  
- un `event_name`,  
- una fecha de evento `event_date`,  
- atributos adicionales (país, canal, etc.).

### 1.2 ¿Por qué usar CTEs (`WITH`) para funnels?

Las CTEs nos permiten:

- Definir **bloques reutilizables** por etapa (first_visit, select_item, …).  
- Evitar duplicados con `SELECT DISTINCT user_id`.  
- Encadenar `LEFT JOIN` desde la primera etapa para contar cuántos usuarios siguen vivos en el embudo.  
- Mantener el SQL **legible**: cada parte del funnel queda nombrada.

### 1.3 Retención y días desde el registro

La **retención** responde a:

> “¿Cuántos usuarios siguen activos X días después de su registro?”

Una tabla típica de retención tiene columnas como:

- `user_id`  
- `country`  
- `activity_date`  
- `day_after_signup` (0, 1, 2, …)  
- `active` (1 si el usuario estuvo activo ese día, 0 si no)

Con esto podemos calcular:

- **Usuarios activos acumulados** hasta D7, D14, D21, D28 (cuántos siguen vivos al menos hasta ese día).  
- **% de retención** = usuarios activos acumulados / usuarios únicos.


## 2. Creación de tablas de ejemplo (SQLiteOnline)

Primero creamos dos tablas pequeñas para practicar:

1. `mercadolibre_funnel` — eventos de funnel por usuario / país.  
2. `mercadolibre_retention` — actividad diaria por usuario / país.

Los datos son **miniaturas** para clase: suficientes para ver la lógica, pero fáciles de revisar a mano.


In [ ]:
-- ==============================================
-- DDL para tablas de práctica (SQLite)
-- Ejecuta este bloque primero en SQLiteOnline.
-- ==============================================

DROP TABLE IF EXISTS mercadolibre_funnel;
DROP TABLE IF EXISTS mercadolibre_retention;

-- Tabla de eventos del funnel
CREATE TABLE mercadolibre_funnel (
  user_id     INTEGER NOT NULL,
  country     TEXT    NOT NULL,   -- país del usuario
  event_name  TEXT    NOT NULL,   -- nombre del evento del funnel
  event_date  TEXT    NOT NULL    -- fecha del evento (YYYY-MM-DD)
);

-- Tabla de actividad / retención
CREATE TABLE mercadolibre_retention (
  user_id          INTEGER NOT NULL,
  country          TEXT    NOT NULL,
  activity_date    TEXT    NOT NULL,   -- día de actividad (YYYY-MM-DD)
  day_after_signup INTEGER NOT NULL,   -- días desde el signup (0,1,2,...)
  active           INTEGER NOT NULL    -- 1 si activo ese día, 0 si no
);


In [ ]:
-- ==============================================
-- INSERTs de ejemplo para mercadolibre_funnel
-- Rango de fechas: 2025-01-01 a 2025-08-31
-- Usuarios repartidos por CO, MX y AR.
-- ==============================================

INSERT INTO mercadolibre_funnel (user_id, country, event_name, event_date) VALUES
  -- Usuario 1 (CO): recorre todo el funnel
  (1, 'CO', 'first_visit',       '2025-01-05'),
  (1, 'CO', 'select_item',       '2025-01-05'),
  (1, 'CO', 'add_to_cart',       '2025-01-06'),
  (1, 'CO', 'begin_checkout',    '2025-01-06'),
  (1, 'CO', 'add_shipping_info', '2025-01-06'),
  (1, 'CO', 'add_payment_info',  '2025-01-07'),
  (1, 'CO', 'purchase',          '2025-01-07'),

  -- Usuario 2 (CO): abandona en add_to_cart
  (2, 'CO', 'first_visit',    '2025-02-10'),
  (2, 'CO', 'select_item',    '2025-02-10'),
  (2, 'CO', 'add_to_cart',    '2025-02-11'),

  -- Usuario 3 (CO): llega hasta begin_checkout pero no compra
  (3, 'CO', 'first_visit',    '2025-03-01'),
  (3, 'CO', 'select_item',    '2025-03-01'),
  (3, 'CO', 'add_to_cart',    '2025-03-02'),
  (3, 'CO', 'begin_checkout', '2025-03-02'),

  -- Usuario 4 (MX): recorre casi todo el funnel, falla en pago
  (4, 'MX', 'first_visit',       '2025-04-15'),
  (4, 'MX', 'select_item',       '2025-04-15'),
  (4, 'MX', 'add_to_cart',       '2025-04-16'),
  (4, 'MX', 'begin_checkout',    '2025-04-16'),
  (4, 'MX', 'add_shipping_info', '2025-04-16'),
  (4, 'MX', 'add_payment_info',  '2025-04-17'),

  -- Usuario 5 (MX): solo navega, nunca agrega al carrito
  (5, 'MX', 'first_visit',  '2025-05-01'),
  (5, 'MX', 'select_item',  '2025-05-01'),

  -- Usuario 6 (AR): primera visita + selección + compra completa
  (6, 'AR', 'first_visit',       '2025-06-20'),
  (6, 'AR', 'select_item',       '2025-06-20'),
  (6, 'AR', 'add_to_cart',       '2025-06-21'),
  (6, 'AR', 'begin_checkout',    '2025-06-21'),
  (6, 'AR', 'add_shipping_info', '2025-06-21'),
  (6, 'AR', 'add_payment_info',  '2025-06-21'),
  (6, 'AR', 'purchase',          '2025-06-22'),

  -- Usuario 7 (AR): llega hasta shipping y abandona
  (7, 'AR', 'first_visit',       '2025-07-10'),
  (7, 'AR', 'select_item',       '2025-07-10'),
  (7, 'AR', 'add_to_cart',       '2025-07-11'),
  (7, 'AR', 'begin_checkout',    '2025-07-11'),
  (7, 'AR', 'add_shipping_info', '2025-07-11');


In [ ]:
-- ==============================================
-- INSERTs de ejemplo para mercadolibre_retention
-- Mini cohorte por país, días 0 a 30 aprox.
-- ==============================================

INSERT INTO mercadolibre_retention (user_id, country, activity_date, day_after_signup, active) VALUES
  -- Usuarios de CO (2 usuarios: 101, 102)
  (101, 'CO', '2025-01-01',  0, 1),
  (101, 'CO', '2025-01-02',  1, 1),
  (101, 'CO', '2025-01-05',  4, 1),
  (101, 'CO', '2025-01-10',  9, 1),
  (101, 'CO', '2025-01-20', 19, 1),
  (101, 'CO', '2025-01-30', 29, 1),

  (102, 'CO', '2025-01-03',  0, 1),
  (102, 'CO', '2025-01-04',  1, 1),
  (102, 'CO', '2025-01-08',  5, 1),
  (102, 'CO', '2025-01-15', 12, 1),
  (102, 'CO', '2025-01-18', 15, 1),

  -- Usuarios de MX (2 usuarios: 201, 202)
  (201, 'MX', '2025-02-01',  0, 1),
  (201, 'MX', '2025-02-02',  1, 1),
  (201, 'MX', '2025-02-04',  3, 1),
  (201, 'MX', '2025-02-09',  8, 1),
  (201, 'MX', '2025-02-12', 11, 1),
  (201, 'MX', '2025-02-20', 19, 1),

  (202, 'MX', '2025-02-03',  0, 1),
  (202, 'MX', '2025-02-05',  2, 1),
  (202, 'MX', '2025-02-10',  7, 1),
  (202, 'MX', '2025-02-17', 14, 1),
  (202, 'MX', '2025-02-25', 22, 1),

  -- Usuarios de AR (2 usuarios: 301, 302)
  (301, 'AR', '2025-03-01',  0, 1),
  (301, 'AR', '2025-03-02',  1, 1),
  (301, 'AR', '2025-03-08',  7, 1),
  (301, 'AR', '2025-03-15', 14, 1),
  (301, 'AR', '2025-03-22', 21, 1),
  (301, 'AR', '2025-03-29', 28, 1),

  (302, 'AR', '2025-03-05',  0, 1),
  (302, 'AR', '2025-03-06',  1, 1),
  (302, 'AR', '2025-03-12',  7, 1),
  (302, 'AR', '2025-03-19', 14, 1),
  (302, 'AR', '2025-03-26', 21, 1);


## 3. Funnels con CTEs por etapa

En esta sección replicamos la lógica de los ejemplos de embudo, pero con las tablas
que acabamos de crear. Nos centraremos en el rango de fechas:

- `2025-01-01` → `2025-08-31`.


### Ejercicio 3.1 — Crear CTEs por etapa y contar usuarios por embudo

**Objetivo**  
Construir CTEs por evento/etapa y obtener los **conteos de usuarios** en cada etapa
del funnel, asegurándonos de que todos parten de `first_visit`.

**Fundamento teórico**

- Cada CTE representa el **conjunto de usuarios únicos** que tuvo ese evento.  
- Al encadenar `LEFT JOIN` desde `first_visit` mantenemos la población inicial y vemos
  cuántos sobreviven en cada etapa.  
- Contar `COUNT(alias.user_id)` en el SELECT final nos da los usuarios en cada etapa.

**Instrucciones**

1. Crea una CTE por evento en `mercadolibre_funnel` en el rango de fechas:
   - `event_date BETWEEN '2025-01-01' AND '2025-08-31'`.  
2. Usa nombres consistentes para las CTEs:  
   `first_visit`, `select_item`, `add_to_cart`, `begin_checkout`, `add_shipping_info`,
   `add_payment_info`, `purchase`.  
3. En cada CTE:
   - `SELECT DISTINCT user_id FROM mercadolibre_funnel ...` para evitar duplicados.  
4. En el query final:
   - Comienza con `FROM first_visit fv`.  
   - Encadena `LEFT JOIN` con cada CTE por `user_id`.  
   - En el `SELECT`, usa `COUNT(alias.user_id)` para cada etapa.  
5. Usa estos alias de salida:  
   `usuarios_first_visit`, `usuarios_select_item`, `usuarios_add_to_cart`,
   `usuarios_begin_checkout`, `usuarios_add_shipping_info`,
   `usuarios_add_payment_info`, `usuarios_purchase`.

**Pistas**

- Recuerda que con `LEFT JOIN` los usuarios que no llegan a una etapa quedan con `NULL` en esa CTE.  
- `COUNT(alias.user_id)` **no cuenta NULL**, así vemos cuántos sí llegaron.


In [ ]:
-- Solución sugerida 3.1 — Conteos por etapa del embudo
WITH first_visit AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'first_visit'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
select_item AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name IN ('select_item', 'select_promotion')
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
add_to_cart AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'add_to_cart'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
begin_checkout AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'begin_checkout'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
add_shipping_info AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'add_shipping_info'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
add_payment_info AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'add_payment_info'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
purchase AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'purchase'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
)
SELECT
  COUNT(fv.user_id)  AS usuarios_first_visit,
  COUNT(si.user_id)  AS usuarios_select_item,
  COUNT(a.user_id)   AS usuarios_add_to_cart,
  COUNT(bc.user_id)  AS usuarios_begin_checkout,
  COUNT(asi.user_id) AS usuarios_add_shipping_info,
  COUNT(api.user_id) AS usuarios_add_payment_info,
  COUNT(p.user_id)   AS usuarios_purchase
FROM first_visit fv
LEFT JOIN select_item      si  ON fv.user_id = si.user_id
LEFT JOIN add_to_cart      a   ON fv.user_id = a.user_id
LEFT JOIN begin_checkout   bc  ON fv.user_id = bc.user_id
LEFT JOIN add_shipping_info asi ON fv.user_id = asi.user_id
LEFT JOIN add_payment_info api ON fv.user_id = api.user_id
LEFT JOIN purchase         p   ON fv.user_id = p.user_id;


### Ejercicio 3.2 — Calcular conversiones desde first_visit a cada etapa

**Objetivo**  
Tomar los conteos por etapa y calcular el **porcentaje de conversión** desde la
primera visita hacia cada etapa posterior.

**Fundamento teórico**

- `conversion_etapa = usuarios_etapa * 100.0 / usuarios_first_visit`.  
- Usamos `NULLIF(usuarios_first_visit, 0)` para evitar división por cero.  
- Redondeamos a 2 decimales para facilitar la lectura.

**Instrucciones**

1. Reutiliza la lógica del ejercicio anterior para crear un CTE `funnel_counts` con los conteos.  
2. En el `SELECT` final, crea 6 columnas de conversión:
   - `conversion_select_item`  
   - `conversion_add_to_cart`  
   - `conversion_begin_checkout`  
   - `conversion_add_shipping_info`  
   - `conversion_add_payment_info`  
   - `conversion_purchase`  
3. Usa `ROUND(..., 2)` para dejar 2 decimales.

**Pistas**

- Puedes envolver el CTE anterior y no necesitas agrupar (es una sola fila).  
- La idea es que el resultado sea una fila con solo columnas de conversión (%).


In [ ]:
-- Solución sugerida 3.2 — Conversiones % desde first_visit
WITH first_visit AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'first_visit'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
select_item AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name IN ('select_item', 'select_promotion')
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
add_to_cart AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'add_to_cart'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
begin_checkout AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'begin_checkout'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
add_shipping_info AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'add_shipping_info'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
add_payment_info AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'add_payment_info'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
purchase AS (
  SELECT DISTINCT user_id
  FROM mercadolibre_funnel
  WHERE event_name = 'purchase'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
funnel_counts AS (
  SELECT
    COUNT(fv.user_id)  AS usuarios_first_visit,
    COUNT(si.user_id)  AS usuarios_select_item,
    COUNT(a.user_id)   AS usuarios_add_to_cart,
    COUNT(bc.user_id)  AS usuarios_begin_checkout,
    COUNT(asi.user_id) AS usuarios_add_shipping_info,
    COUNT(api.user_id) AS usuarios_add_payment_info,
    COUNT(p.user_id)   AS usuarios_purchase
  FROM first_visit fv
  LEFT JOIN select_item      si  ON fv.user_id = si.user_id
  LEFT JOIN add_to_cart      a   ON fv.user_id = a.user_id
  LEFT JOIN begin_checkout   bc  ON fv.user_id = bc.user_id
  LEFT JOIN add_shipping_info asi ON fv.user_id = asi.user_id
  LEFT JOIN add_payment_info api ON fv.user_id = api.user_id
  LEFT JOIN purchase         p   ON fv.user_id = p.user_id
)
SELECT
  ROUND(usuarios_select_item      * 100.0 / NULLIF(usuarios_first_visit, 0), 2)
    AS conversion_select_item,
  ROUND(usuarios_add_to_cart      * 100.0 / NULLIF(usuarios_first_visit, 0), 2)
    AS conversion_add_to_cart,
  ROUND(usuarios_begin_checkout   * 100.0 / NULLIF(usuarios_first_visit, 0), 2)
    AS conversion_begin_checkout,
  ROUND(usuarios_add_shipping_info * 100.0 / NULLIF(usuarios_first_visit, 0), 2)
    AS conversion_add_shipping_info,
  ROUND(usuarios_add_payment_info * 100.0 / NULLIF(usuarios_first_visit, 0), 2)
    AS conversion_add_payment_info,
  ROUND(usuarios_purchase         * 100.0 / NULLIF(usuarios_first_visit, 0), 2)
    AS conversion_purchase
FROM funnel_counts;


### Ejercicio 3.3 — Embudo segmentado por país

**Objetivo**  
Calcular conversiones del embudo **por país**, para detectar en qué etapa se pierden
más usuarios en cada mercado.

**Fundamento teórico**

- Propagamos `country` en cada CTE para poder agrupar el funnel por país.  
- En el CTE `funnel_counts` contamos usuarios por (país, etapa).  
- Luego calculamos conversiones sobre `usuarios_first_visits` de ese país.

**Instrucciones**

1. En cada CTE (`first_visits`, `select_item`, etc.):  
   - Selecciona `DISTINCT user_id, country`.  
   - Mantén el filtro de fechas.  
2. En `funnel_counts`:
   - Selecciona `fv.country`.  
   - Cuenta **usuarios únicos** por etapa con `COUNT(DISTINCT alias.user_id)`.  
   - Une por `user_id` **y** `country`.  
   - Agrupa por `fv.country`.  
3. En el `SELECT` final:  
   - Devuelve `country` y las conversiones % por etapa.  
   - Ordena por `conversion_purchase` descendente.

**Pistas**

- Usa `NULLIF(usuarios_first_visits, 0)` para evitar divisiones por cero.  
- Observa qué país tiene mejor `conversion_purchase`.


In [ ]:
-- Solución sugerida 3.3 — Conversiones de funnel por país
WITH first_visits AS (
  SELECT DISTINCT user_id, country
  FROM mercadolibre_funnel
  WHERE event_name = 'first_visit'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
select_item AS (
  SELECT DISTINCT user_id, country
  FROM mercadolibre_funnel
  WHERE event_name IN ('select_item', 'select_promotion')
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
add_to_cart AS (
  SELECT DISTINCT user_id, country
  FROM mercadolibre_funnel
  WHERE event_name = 'add_to_cart'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
begin_checkout AS (
  SELECT DISTINCT user_id, country
  FROM mercadolibre_funnel
  WHERE event_name = 'begin_checkout'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
add_shipping_info AS (
  SELECT DISTINCT user_id, country
  FROM mercadolibre_funnel
  WHERE event_name = 'add_shipping_info'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
add_payment_info AS (
  SELECT DISTINCT user_id, country
  FROM mercadolibre_funnel
  WHERE event_name = 'add_payment_info'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
purchase AS (
  SELECT DISTINCT user_id, country
  FROM mercadolibre_funnel
  WHERE event_name = 'purchase'
    AND event_date BETWEEN '2025-01-01' AND '2025-08-31'
),
funnel_counts AS (
  SELECT
    fv.country,
    COUNT(DISTINCT fv.user_id)  AS usuarios_first_visits,
    COUNT(DISTINCT si.user_id)  AS usuarios_select_item,
    COUNT(DISTINCT a.user_id)   AS usuarios_add_to_cart,
    COUNT(DISTINCT bc.user_id)  AS usuarios_begin_checkout,
    COUNT(DISTINCT asi.user_id) AS usuarios_add_shipping_info,
    COUNT(DISTINCT api.user_id) AS usuarios_add_payment_info,
    COUNT(DISTINCT p.user_id)   AS usuarios_purchase
  FROM first_visits fv
  LEFT JOIN select_item      si  ON fv.user_id = si.user_id AND fv.country = si.country
  LEFT JOIN add_to_cart      a   ON fv.user_id = a.user_id  AND fv.country = a.country
  LEFT JOIN begin_checkout   bc  ON fv.user_id = bc.user_id AND fv.country = bc.country
  LEFT JOIN add_shipping_info asi ON fv.user_id = asi.user_id AND fv.country = asi.country
  LEFT JOIN add_payment_info api ON fv.user_id = api.user_id AND fv.country = api.country
  LEFT JOIN purchase         p   ON fv.user_id = p.user_id  AND fv.country = p.country
  GROUP BY fv.country
)
SELECT
  country,
  usuarios_first_visits,
  ROUND(usuarios_select_item      * 100.0 / NULLIF(usuarios_first_visits, 0), 2)
    AS conversion_select_item,
  ROUND(usuarios_add_to_cart      * 100.0 / NULLIF(usuarios_first_visits, 0), 2)
    AS conversion_add_to_cart,
  ROUND(usuarios_begin_checkout   * 100.0 / NULLIF(usuarios_first_visits, 0), 2)
    AS conversion_begin_checkout,
  ROUND(usuarios_add_shipping_info * 100.0 / NULLIF(usuarios_first_visits, 0), 2)
    AS conversion_add_shipping_info,
  ROUND(usuarios_add_payment_info * 100.0 / NULLIF(usuarios_first_visits, 0), 2)
    AS conversion_add_payment_info,
  ROUND(usuarios_purchase         * 100.0 / NULLIF(usuarios_first_visits, 0), 2)
    AS conversion_purchase
FROM funnel_counts
ORDER BY conversion_purchase DESC;


## 4. Retención: usuarios activos acumulados y % por país

Ahora cambiamos de tabla (`mercadolibre_retention`) para practicar métricas de
**retención** en diferentes hitos de días: D7, D14, D21 y D28.


### Ejercicio 4.1 — Usuarios activos acumulados por país (D7, D14, D21, D28)

**Objetivo**  
Para cada país, contar **usuarios activos acumulados** desde su registro hasta
los días 7, 14, 21 y 28, en el rango `2025-01-01` → `2025-08-31`.

**Fundamento teórico**

- Un usuario cuenta como "activo acumulado a D7" si:
  - `active = 1`  
  - y `day_after_signup >= 7` (algún día con actividad al menos hasta el día 7).  
- Usamos `COUNT(DISTINCT CASE WHEN ... THEN user_id END)` para evitar duplicados.

**Instrucciones**

1. Trabaja desde `mercadolibre_retention`.  
2. Filtra por `activity_date BETWEEN '2025-01-01' AND '2025-08-31'`.  
3. Para cada país (`GROUP BY country`):  
   - Crea columnas:
     - `users_d7`, `users_d14`, `users_d21`, `users_d28`.  
   - Cada una con `COUNT(DISTINCT CASE WHEN day_after_signup >= X AND active = 1 THEN user_id END)`.  
4. Ordena por `country`.

**Pistas**

- El patrón es siempre el mismo; solo cambia el umbral (`>= 7`, `>= 14`, etc.).  
- Recuerda el orden de cláusulas: `SELECT`, `FROM`, `WHERE`, `GROUP BY`, `ORDER BY`.


In [ ]:
-- Solución sugerida 4.1 — Usuarios activos acumulados por país
SELECT
  country,
  COUNT(DISTINCT CASE WHEN day_after_signup >= 7  AND active = 1 THEN user_id END)
    AS users_d7,
  COUNT(DISTINCT CASE WHEN day_after_signup >= 14 AND active = 1 THEN user_id END)
    AS users_d14,
  COUNT(DISTINCT CASE WHEN day_after_signup >= 21 AND active = 1 THEN user_id END)
    AS users_d21,
  COUNT(DISTINCT CASE WHEN day_after_signup >= 28 AND active = 1 THEN user_id END)
    AS users_d28
FROM mercadolibre_retention
WHERE activity_date BETWEEN '2025-01-01' AND '2025-08-31'
GROUP BY country
ORDER BY country;


### Ejercicio 4.2 — Convertir conteos a % de retención por país

**Objetivo**  
Tomar los conteos de usuarios activos acumulados y convertirlos en **porcentajes
de retención** al día 7, 14, 21 y 28 por país.

**Fundamento teórico**

- Para cada país:
  - `retention_d7_pct = users_d7 * 100.0 / total_usuarios_unicos`.  
- Donde `total_usuarios_unicos` se calcula con `COUNT(DISTINCT user_id)` en ese país.  
- Usamos `NULLIF(..., 0)` para evitar división por cero.  
- Redondeamos a 1 decimal.

**Instrucciones**

1. Parte del mismo `FROM` / `WHERE` que el ejercicio 4.1.  
2. Para cada hito (7, 14, 21, 28):  
   - Usa el mismo `CASE WHEN` que en 4.1, pero ahora dentro de `ROUND(100.0 * conteo / NULLIF(total, 0), 1)`.  
3. Usa alias: `retention_d7_pct`, `retention_d14_pct`, `retention_d21_pct`, `retention_d28_pct`.  
4. Mantén el `GROUP BY country` y `ORDER BY country`.

**Pistas**

- El denominador es el mismo para todas las columnas: `COUNT(DISTINCT user_id)`.  
- Piensa que estás respondiendo: “¿Qué % de la cohorte sigue activo al menos hasta ese día?”.


In [ ]:
-- Solución sugerida 4.2 — % de retención por país
SELECT
  country,
  ROUND(
    100.0 * COUNT(DISTINCT CASE WHEN day_after_signup >= 7  AND active = 1 THEN user_id END)
    / NULLIF(COUNT(DISTINCT user_id), 0),
    1
  ) AS retention_d7_pct,
  ROUND(
    100.0 * COUNT(DISTINCT CASE WHEN day_after_signup >= 14 AND active = 1 THEN user_id END)
    / NULLIF(COUNT(DISTINCT user_id), 0),
    1
  ) AS retention_d14_pct,
  ROUND(
    100.0 * COUNT(DISTINCT CASE WHEN day_after_signup >= 21 AND active = 1 THEN user_id END)
    / NULLIF(COUNT(DISTINCT user_id), 0),
    1
  ) AS retention_d21_pct,
  ROUND(
    100.0 * COUNT(DISTINCT CASE WHEN day_after_signup >= 28 AND active = 1 THEN user_id END)
    / NULLIF(COUNT(DISTINCT user_id), 0),
    1
  ) AS retention_d28_pct
FROM mercadolibre_retention
WHERE activity_date BETWEEN '2025-01-01' AND '2025-08-31'
GROUP BY country
ORDER BY country;


## 5. Cierre y siguientes pasos

En este webinar:

- Practicaste cómo **armar un funnel** por etapas usando CTEs y `LEFT JOIN`.  
- Calculaste **conteos y conversiones %** desde la primera visita.  
- Segmentaste el embudo por país para identificar **dónde se pierden usuarios**.  
- Calculaste **usuarios activos acumulados** a D7, D14, D21 y D28.  
- Transformaste esos conteos en **porcentajes de retención por país**.

### 5.1 Qué deberías llevarte

1. La mecánica de construir funnels con CTEs es siempre la misma:  
   - bloques por etapa → `LEFT JOIN` → conteos → conversiones.  
2. Retención y funnels son dos caras de la misma moneda:  
   - funnels = progresión dentro de una sesión / journey.  
   - retención = progreso en el tiempo después del signup.  
3. Tanto en funnels como en retención, **segmentar** (por país, canal, etc.)
   revela insights accionables.

### 5.2 Siguientes pasos sugeridos

- Extender el embudo con más atributos (dispositivo, categoría de producto).  
- Crear un funnel mensual (CTEs + `WHERE event_date` por mes).  
- Cruzar retención y funnels:  
  - ¿Los usuarios que completan `purchase` tienen mejor retención que quienes no compran?  
- Preparar una breve nota ejecutiva con 3–5 bullets por país:
  - Punto fuerte del funnel.  
  - Etapa más débil.  
  - Nivel de retención D28 y posibles acciones.
